In [1]:
import math
from typing import List, Tuple, Union
import numpy as np

A -- PARSING

In [ ]:
def parse_tsplib_file(filename: str) -> List[Tuple[float, float]]:

    coords = []
    dimension = None
    edge_weight_type = None
    in_node_section = False
    
    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            
            # Saltar líneas vacías, comentarios y EOF
            if not line or line.startswith('COMMENT') or line == 'EOF':
                continue
            
            # Obtener dimensión
            if line.startswith('DIMENSION'):
                dimension = int(line.split(':')[1].strip())
            
            # Validar tipo de distancia
            if line.startswith('EDGE_WEIGHT_TYPE'):
                edge_weight_type = line.split(':')[1].strip()
                if edge_weight_type != 'EUC_2D':
                    raise ValueError(f"Se requiere EDGE_WEIGHT_TYPE=EUC_2D, no {edge_weight_type}")
            
            # Inicio de sección de coordenadas
            if line == 'NODE_COORD_SECTION':
                in_node_section = True
                continue
            
            # Procesar coordenadas
            if in_node_section:
                parts = line.split()
                if len(parts) >= 3:
                    try:
                        x = float(parts[1])
                        y = float(parts[2])
                        coords.append((x, y))
                    except ValueError:
                        raise ValueError(f"Formato de coordenada inválido: {line}")
    
    # Validaciones finales
    if edge_weight_type != 'EUC_2D':
        raise ValueError("El archivo debe especificar EDGE_WEIGHT_TYPE=EUC_2D")
    
    if dimension is not None and len(coords) != dimension:
        raise ValueError(f"Dimensión especificada ({dimension}) no coincide con coordenadas leídas ({len(coords)})")
    
    return coords

B -- DISTANCIAS

In [ ]:
def calcular_matriz_distancias(coords: List[Tuple[float, float]]) -> np.ndarray:
    n = len(coords)
    D = np.zeros((n, n), dtype=int)
    
    for i in range(n):
        xi, yi = coords[i]
        for j in range(i + 1, n):
            xj, yj = coords[j]
            dx = xi - xj
            dy = yi - yj
            distancia = math.sqrt(dx*dx + dy*dy)
            distancia_redondeada = int(math.floor(distancia + 0.5))
            D[i, j] = distancia_redondeada
            D[j, i] = distancia_redondeada
    
    return D

C -- REPRESEMTACION

In [ ]:
def crear_permutacion_aleatoria(n: int) -> List[int]:
    permutacion = list(range(n))
    np.random.shuffle(permutacion)
    return permutacion

def validar_permutacion(permutacion: List[int], n: int) -> bool: #con n elementos
    if len(permutacion) != n:
        return False
    return sorted(permutacion) == list(range(n))

D -- FITNESS

In [ ]:
def calcular_costo_tour(permutacion: List[int], D: np.ndarray) -> int:
    n = len(permutacion)
    costo = 0
    
    for k in range(n - 1):
        i = permutacion[k]
        j = permutacion[k + 1]
        costo += D[i, j]
   
    costo += D[permutacion[-1], permutacion[0]]
    return costo

def calcular_fitness(permutacion: List[int], D: np.ndarray, epsilon: float = 1.0) -> float:
    costo = calcular_costo_tour(permutacion, D)
    return 1.0 / (epsilon + costo)